# Real Data

In [ ]:
import sys
import os
import pickle
from tqdm import tqdm
from topolearn import *
import warnings
from sklearn.model_selection import train_test_split
import scipy.io
import numpy as np
warnings.filterwarnings("ignore")

J=2
P=3

seed = 0
mat = scipy.io.loadmat('C:\\Users\\engri\\Desktop\\tesi\\TSP-DictionaryLearning\\real_data\\data_real.mat')
edge_signals = np.array(mat["signal_edge"].T, dtype=float)
valid_signal = np.where(np.sum(edge_signals, axis=1)!=0)
edge_signals = edge_signals[valid_signal]
Y_train, Y_test = train_test_split(edge_signals, test_size=0.2, random_state=seed)
Y_train = Y_train.T
Y_test = Y_test.T
Lu = mat["B2"] @ mat["B2"].T
Ld = mat["B1"].T @ mat["B1"]
L=Lu+Ld
Lu = np.array(Lu, dtype=float)
Ld = np.array(Ld, dtype=float)
L = np.array(L, dtype=float)


with open("C:\\Users\\engri\\Desktop\\tesi\\TSP-DictionaryLearning\\synthetic_data\\random_sparsity25\\full_data_separated.pkl", "rb") as file:
    data = pickle.load(file)
c = data["c_true"].max()
epsilon = data["epsilon_true"].mean()

K0_coll = np.arange(5, 46, 4)

init_params = {
    "J": J,
    "P": P,
    "true_prob_T": 1.,
    "n": mat["B1"].shape[0],
    "sparsity": 25,
    "B1_true": mat["B1"],
    "B2_true": mat["B2"]
}

algo_params = {
    "lambda_": 1e-7,
    "tol": 1e-7,
    "patience": 5,
    "max_iter": 100,
    "QP": True,
    "mode": "optimistic",
    "warmup": 2,
    "on_test": False,
    "complete": True,
    "verbose": True,
}

min_error_cfou_train = np.zeros((len(K0_coll)))
min_error_cfou_test = np.zeros((len(K0_coll)))
min_error_fou_train = np.zeros((len(K0_coll)))
min_error_fou_test = np.zeros((len(K0_coll)))
min_error_slep_train = np.zeros((len(K0_coll)))
min_error_slep_test = np.zeros((len(K0_coll)))
min_error_wave_train = np.zeros((len(K0_coll)))
min_error_wave_test = np.zeros((len(K0_coll)))
min_error_sep_train = np.zeros((len(K0_coll)))
min_error_sep_test = np.zeros((len(K0_coll)))
min_error_complete_train = np.zeros((len(K0_coll)))
min_error_complete_test = np.zeros((len(K0_coll)))
min_error_pess_train = np.zeros((len(K0_coll)))
min_error_pess_test = np.zeros((len(K0_coll)))
approx_cfou = np.zeros((len(K0_coll)))
approx_fou = np.zeros((len(K0_coll)))
approx_sep = np.zeros((len(K0_coll)))
approx_slep = np.zeros((len(K0_coll)))
approx_wave = np.zeros((len(K0_coll)))
approx_comp = np.zeros((len(K0_coll)))
approx_pess = np.zeros((len(K0_coll)))

dict_errors = {
    "classic_fourier": (min_error_cfou_train, min_error_cfou_test, approx_cfou),
    "fourier": (min_error_fou_train, min_error_fou_test, approx_fou),
    "slepians": (min_error_slep_train, min_error_slep_test, approx_slep),
    "wavelet": (min_error_wave_train, min_error_wave_test, approx_wave),
    "separated": (min_error_sep_train, min_error_sep_test, approx_sep),
    "complete": (min_error_complete_train, min_error_complete_test, approx_comp),
    "complete_pess": (min_error_pess_train, min_error_pess_test, approx_pess)
}

dict_errors2 = {
    "classic_fourier": min_error_cfou_test,
    "fourier": min_error_fou_test,
    "slepians": min_error_slep_test,
    "wavelet": min_error_wave_test,
    "separated": min_error_sep_test,
    "complete": min_error_complete_test,
    "complete_pess": min_error_pess_test,
}

dict_types = {
    "classic_fourier": ("Fourier", "classic_fourier"),
    "fourier": ("Topological Fourier", "fourier"),
    "slepians": ("Topological Slepians", "slepians"),
    "wavelet": ("Hodgelet", "wavelet"),
    "separated": ("Separated Hodge Laplacian", "separated"),
    "complete": ("Separated Hodge Laplacian with Topology learning","separated"),
    "complete_pess": ("Separated Hodge Laplacian with Pessimistic Topology learning","separated"),
}

models = {}

for k0_index, k0 in tqdm(enumerate(K0_coll), leave=False):

    for d in dict_types.items():

        model = TopoSolver(
            X_train=None,
            X_test=None,
            Y_train=Y_train,
            Y_test=Y_test,
            c=c,
            epsilon=epsilon,
            K0=k0,
            dictionary_type=d[1][1],
            **init_params,
        )

        learn_topology = True if "complete" in d[0] else False
        if "pess" in d[0]:
            algo_params["mode"]="pessimistic"
        else:
            algo_params["mode"]="optimistic"

        (
            dict_errors[d[0]][0][k0_index],
            dict_errors[d[0]][1][k0_index],
            dict_errors[d[0]][2][k0_index],
        ) = model.fit(
            Lu_true=Lu,
            init_mode="only_X",
            learn_topology=learn_topology,
            **algo_params,
        )
        dict_errors2[d[0]] = dict_errors[d[0]][1]
        models[f"{k0_index},{d[0]}"] = model

        print(
            f"Sparsity: {k0} Testing {d[1][0]}... Done! Test Error: {dict_errors[d[0]][1][k0_index]:.6f}"
        )

with open(f"C:\\Users\\engri\\Desktop\\tesi\\TSP-DictionaryLearning\\results\\real\\real_J{J}P{P}.pkl", "wb") as file:
    pickle.dump(models, file)
    pickle.dump(dict_errors, file)



0it [00:00, ?it/s]

Sparsity: 5 Testing Fourier... Done! Test Error: 0.777496
Sparsity: 5 Testing Topological Fourier... Done! Test Error: 0.610367
Sparsity: 5 Testing Topological Slepians... Done! Test Error: 0.188351
Sparsity: 5 Testing Hodgelet... Done! Test Error: 0.244725
Sparsity: 5 Testing Separated Hodge Laplacian... Done! Test Error: 0.062374
Removing 1 triangles from the topology... 
 ... The min error: 50.000 !
Removing 2 triangles from the topology... 
 ... The min error: 49.000 !
Removing 3 triangles from the topology... 
 ... The min error: 0.049 !
Removing 4 triangles from the topology... 
 ... The min error: 0.044 !
Removing 5 triangles from the topology... 
 ... The min error: 0.043 !
Removing 6 triangles from the topology... 
 ... The min error: 0.042 !
Removing 7 triangles from the topology... 
 ... The min error: 0.042 !
Removing 8 triangles from the topology... 
 ... The min error: 0.042 !
Removing 9 triangles from the topology... 
 ... The min error: 0.042 !
Removing 10 triangles fro

1it [00:21, 21.81s/it]

Sparsity: 5 Testing Separated Hodge Laplacian with Pessimistic Topology learning... Done! Test Error: 0.073212
Sparsity: 9 Testing Fourier... Done! Test Error: 0.711368
Sparsity: 9 Testing Topological Fourier... Done! Test Error: 0.481200
Sparsity: 9 Testing Topological Slepians... Done! Test Error: 0.083591
Sparsity: 9 Testing Hodgelet... Done! Test Error: 0.086897
Sparsity: 9 Testing Separated Hodge Laplacian... Done! Test Error: 0.010195
Removing 1 triangles from the topology... 
 ... The min error: 7.000 !
Removing 2 triangles from the topology... 
 ... The min error: 7.000 !
Removing 3 triangles from the topology... 
 ... The min error: 0.007 !
Removing 4 triangles from the topology... 
 ... The min error: 0.007 !
Removing 5 triangles from the topology... 
 ... The min error: 0.007 !
Removing 6 triangles from the topology... 
 ... The min error: 0.007 !
Removing 7 triangles from the topology... 
 ... The min error: 0.007 !
Removing 8 triangles from the topology... 
 ... The min er

2it [00:33, 16.12s/it]

...

9it [04:57, 44.43s/it]

Sparsity: 37 Testing Separated Hodge Laplacian with Pessimistic Topology learning... Done! Test Error: 0.000019
Sparsity: 41 Testing Fourier... Done! Test Error: 0.590043
Sparsity: 41 Testing Topological Fourier... Done! Test Error: 0.065235
Sparsity: 41 Testing Topological Slepians... Done! Test Error: 0.000648
Sparsity: 41 Testing Hodgelet... Done! Test Error: 0.000733
Sparsity: 41 Testing Separated Hodge Laplacian... Done! Test Error: 0.000081
Removing 1 triangles from the topology... 
 ... The min error: 0.000 !
Removing 2 triangles from the topology... 
 ... The min error: 0.000 !
Removing 3 triangles from the topology... 
 ... The min error: 0.000 !
Removing 4 triangles from the topology... 
 ... The min error: 0.000 !
Removing 5 triangles from the topology... 
 ... The min error: 0.000 !
Removing 6 triangles from the topology... 
 ... The min error: 0.000 !
Removing 7 triangles from the topology... 
 ... The min error: 0.000 !
Removing 8 triangles from the topology... 
 ... The 

10it [06:23, 57.19s/it]

Sparsity: 41 Testing Separated Hodge Laplacian with Pessimistic Topology learning... Done! Test Error: 0.000006
Sparsity: 45 Testing Fourier... Done! Test Error: 0.589859
Sparsity: 45 Testing Topological Fourier... Done! Test Error: 0.053893
Sparsity: 45 Testing Topological Slepians... Done! Test Error: 0.000333
Sparsity: 45 Testing Hodgelet... Done! Test Error: 0.000411
Sparsity: 45 Testing Separated Hodge Laplacian... Done! Test Error: 0.000046
Removing 1 triangles from the topology... 
 ... The min error: 0.000 !
Removing 2 triangles from the topology... 
 ... The min error: 0.000 !
Removing 3 triangles from the topology... 
 ... The min error: 0.000 !
Removing 4 triangles from the topology... 
 ... The min error: 0.000 !
Sparsity: 45 Testing Separated Hodge Laplacian with Topology learning... Done! Test Error: 0.000033
Adding 1 triangles to the topology... 
 ... The min error: 0.000 !
Adding 2 triangles to the topology... 
 ... The min error: 0.000 !
Adding 3 triangles to the topol

Sparsity: 45 Testing Separated Hodge Laplacian with Pessimistic Topology learning... Done! Test Error: 0.000003


In [7]:
len(K0_coll)

11

# Slepians and Hodgelets